In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
all_data = pd.read_csv("Nomao.data", header = None)
all_data.replace('?', np.NaN, inplace=True)

all_data = all_data.drop(0, axis=1)

data_train, data_test, labels_train, labels_test = train_test_split(all_data.iloc[:,:118], all_data.iloc[:,[118]], test_size=0.3, stratify=all_data.iloc[:,-1])

imp = SimpleImputer(missing_values=np.NaN, strategy="most_frequent")
cat_data_train = list ()
num_data_train = list ()
cat_data_test = list ()
num_data_test = list ()
cat_cols = [7, 8, 15, 16, 23, 24, 31, 32, 39, 40, 47, 48, 55, 56, 63, 64, 71, 72, 79, 80, 87, 88, 92, 96, 100, 104, 108, 112, 116] 
for col in cat_cols :
  cat_data_train.append(pd.DataFrame(imp.fit_transform(data_train.iloc[:,[col-1]])))
  cat_data_test.append(pd.DataFrame(imp.transform(data_test.iloc[:,[col-1]])))
cols = all_data.columns
num_cols = list(set (cols) - set (cat_cols) - {119})
imp = SimpleImputer(missing_values=np.NaN, strategy="mean")
for col in num_cols :
  num_data_train.append(pd.DataFrame(imp.fit_transform(data_train.iloc[:,[col-1]])))
  num_data_test.append(pd.DataFrame(imp.transform(data_test.iloc[:,[col-1]])))

j1 , j2 = 0, 0

idata_train = pd.DataFrame()
idata_test = pd.DataFrame()
for col in range(1,119) :
  if j2 < 29 and cat_cols[j2] == col :
    idata_train = pd.concat([idata_train,cat_data_train[j2]],join="outer",axis=1)
    idata_test = pd.concat([idata_test, cat_data_test[j2]],join="outer",axis=1)
    j2 = j2 + 1
  else :
    idata_train = pd.concat([idata_train,num_data_train[j1]],join="outer",axis=1)
    idata_test = pd.concat([idata_test,num_data_test[j1]],join="outer",axis=1)    
    j1 = j1 + 1

temp = pd.concat([idata_train,idata_test], join="outer", axis=0)
dummy_data = pd.get_dummies(temp, [i-1 for i in cat_cols])
np_dummy_data = dummy_data.apply(pd.to_numeric).values

np_data_train = np_dummy_data[:24125,:] #οι γραμμές δεν αλλάζουν θέση οπότε αφού σειριακά
np_data_test = np_dummy_data[24125:,:]  #ενώσαμε τα train, test σειριακά θα τα χωρίσουμε

mapping = {-1 : 0, +1 : 1}

labels_train[119] = labels_train[119].map(mapping)
labels_test[119] = labels_test[119].map(mapping)

np_labels_train = labels_train[119].values.flatten()
np_labels_test = labels_test[119].values.flatten()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (57,58,59,60,61,62,65,66,67,68,69,70,101,102,103,105,106,107,117) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# ΑΠΟΤΕΛΕΣΜΑΤΑ ΧΩΡΙΣ PREPROCESSING
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.metrics import precision_recall_fscore_support
gnb = GaussianNB()
gnb_model = gnb.fit(np_data_train, np_labels_train)
gnb_pred = gnb_model.predict(np_data_test)
gnb_conf_matrix = confusion_matrix(np_labels_test, gnb_pred)
print("Confusion Maxtrix : \n", gnb_conf_matrix)
print("f1-micro average : ", f1_score(np_labels_test, gnb_pred, average = 'micro'))
print("f1-macro average : ", f1_score(np_labels_test, gnb_pred, average = 'macro'))
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='micro'), "\n")
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='macro'), "\n")
#print(classification_report(np_labels_test, gnb_pred))

Confusion Maxtrix : 
 [[1623 1330]
 [ 490 6897]]
f1-micro average :  0.8239845261121855
f1-macro average :  0.7620900716157076


In [ ]:
from imblearn.pipeline import Pipeline

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler

selector = VarianceThreshold()
scaler = StandardScaler()
ros = RandomOverSampler()
pca = PCA()
gnb = GaussianNB()
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', ros), ('pca', pca), ('gnb', gnb)])
pipe.fit(np_data_train, np_labels_train)
preds = pipe.predict(np_data_test)
print("Confusion Matrix : \n", confusion_matrix(np_labels_test, preds))
print("f1-micro average : ", f1_score(np_labels_test, preds, average = 'micro'))
print("f1-macro average : ", f1_score(np_labels_test, preds, average = 'macro'))
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='micro'), "\n")
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='macro'), "\n")
#print(classification_report(np_labels_test, preds))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Confusion Matrix : 
 [[2848  105]
 [4563 2824]]
f1-micro average :  0.5485493230174081
f1-macro average :  0.548546890846697


In [ ]:
train_var = np_data_train.var(axis=0)
print(train_var)
print(np.max(train_var))

[1.80123600e-01 1.44419926e-01 9.34821310e-02 1.36728339e-01
 1.05745261e-01 1.39032133e-01 4.19849028e-02 4.46005306e-02
 2.22427293e-02 3.28160715e-02 2.65083425e-02 3.32149393e-02
 4.47838900e-02 4.48844645e-02 1.69998247e-02 2.86309162e-02
 1.75026744e-02 2.90851613e-02 4.83444930e-02 4.56985137e-02
 2.85531278e-02 4.25544624e-02 3.19108669e-02 4.77958971e-02
 3.64871044e-03 5.92590814e-03 3.20201298e-03 4.48756099e-03
 5.24804965e-03 7.20819455e-03 3.27358583e-02 3.27358583e-02
 1.15632364e-02 1.89432223e-02 1.17686274e-02 1.89570422e-02
 4.34623677e-02 4.40418090e-02 2.65511673e-02 3.67398451e-02
 2.69947943e-02 3.69069647e-02 6.47477778e-02 6.46881993e-02
 3.72426181e-02 5.53633316e-02 5.08391356e-02 6.61397276e-02
 4.11988408e-02 7.80187987e-02 4.09275335e-02 5.41895795e-02
 5.40109670e-02 7.31678155e-02 6.66104568e-02 6.72537367e-02
 1.82151833e-02 3.77527638e-02 1.68391526e-02 3.77088705e-02
 2.48640276e-04 2.48640276e-04 2.48640276e-04 2.48640276e-04
 2.48640276e-04 2.486402

In [ ]:
#ΑΠΟΤΕΛΕΣΜΑΤΑ ΒΕΛΤΙΣΤΟΥ PIPE
from sklearn.model_selection import GridSearchCV

vthreshold = [0, 0.03, 0.05, 0.06, 0.08, 1e-1]
n_components = [1,5,8,10,15,20]
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('sampler', ros), ('pca', pca), ('gnb', gnb)], memory = 'tmp')
estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components), cv=5, scoring='f1_macro',n_jobs=-1)

In [ ]:
import time
start_time = time.time()
estimator.fit(np_data_train, np_labels_train)
preds = estimator.predict(np_data_test)
print("Συνολικός χρόνος fit και predict: %s seconds" % (time.time()-start_time))
print("Confusion Matrix : \n", confusion_matrix(np_labels_test, preds))
print("f1-micro average : ", f1_score(np_labels_test, preds, average = 'micro'))
print("f1-macro average : ", f1_score(np_labels_test, preds, average = 'macro'))
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='micro'), "\n")
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='macro'), "\n")
#print(classification_report(np_labels_test, preds))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Συνολικός χρόνος fit και predict: 76.33813691139221 seconds
Confusion Matrix : 
 [[2735  218]
 [ 813 6574]]
f1-micro average :  0.9002901353965184
f1-macro average :  0.8843479233188329


In [ ]:
print(estimator.best_estimator_)
print(estimator.best_params_)

Pipeline(memory='tmp',
         steps=[('selector', VarianceThreshold(threshold=0.06)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('sampler',
                 RandomOverSampler(random_state=None, ratio=None,
                                   return_indices=False,
                                   sampling_strategy='auto')),
                ('pca',
                 PCA(copy=True, iterated_power='auto', n_components=20,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('gnb', GaussianNB(priors=None, var_smoothing=1e-09))],
         verbose=False)
{'pca__n_components': 20, 'selector__threshold': 0.06}


In [ ]:
#ΑΠΟΤΕΛΕΣΜΑΤΑ PIPE ΧΩΡΙΣ SCALER
pipe = Pipeline(steps=[('selector', selector), ('sampler', ros), ('pca', pca), ('gnb', gnb)], memory = 'tmp')
estimator = GridSearchCV(pipe, dict(selector__threshold=vthreshold, pca__n_components=n_components), cv=10, scoring='f1_macro',n_jobs=-1)
estimator.fit(np_data_train, np_labels_train)
preds = estimator.predict(np_data_test)
print("Confusion Matrix : \n", confusion_matrix(np_labels_test, preds))
print("f1-micro average : ", f1_score(np_labels_test, preds, average = 'micro'))
print("f1-macro average : ", f1_score(np_labels_test, preds, average = 'macro'))
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='micro'), "\n")
#print(precision_recall_fscore_support(np_labels_test, gnb_pred, average='macro'), "\n")
#print(classification_report(np_labels_test, preds))
print(estimator.best_estimator_)
print(estimator.best_params_)

Confusion Matrix : 
 [[2727  226]
 [ 719 6668]]
f1-micro average :  0.9086073500967118
f1-macro average :  0.8930744191996862
Pipeline(memory='tmp',
         steps=[('selector', VarianceThreshold(threshold=0.03)),
                ('sampler',
                 RandomOverSampler(random_state=None, ratio=None,
                                   return_indices=False,
                                   sampling_strategy='auto')),
                ('pca',
                 PCA(copy=True, iterated_power='auto', n_components=15,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('gnb', GaussianNB(priors=None, var_smoothing=1e-09))],
         verbose=False)
{'pca__n_components': 15, 'selector__threshold': 0.03}


In [ ]:
# MLP CLASSIFIER
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(np_data_train, np_labels_train)
mlp_preds = mlp.predict(np_data_test)
print("Confusion Matrix : \n", confusion_matrix(np_labels_test, mlp_preds))
print("f1-micro average : ", f1_score(np_labels_test, mlp_preds, average = 'micro'))
print("f1-macro average : ", f1_score(np_labels_test, mlp_preds, average = 'macro'))

Confusion Matrix : 
 [[2816  137]
 [ 293 7094]]
f1-micro average :  0.9584139264990329
f1-macro average :  0.9498252629970083
